In [1]:
import numpy as np
import pandas as pd
import datetime
import itertools
import copy
import time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.vector_ar.vecm import coint_johansen #the Johansen cointegration test for determining the cointegration rank of a VECM
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
# adfuller为ADF检验，coint为协整检验
from statsmodels.tsa.stattools import adfuller, coint #the augmented Engle-Granger two-step cointegration test
import statsmodels.stats.diagnostic

In [2]:
# 通过警告过滤器进行控制是否发出警告消息
import warnings
warnings.filterwarnings("ignore")

## 1.导入数据&处理变量

In [118]:
# 从Excel导入数据
# 运行之前请修改下一行的文件路径
filepath = r'通用行业模型结果_iris_v0.4-自动化测算-量纲统一(1).xlsx'
def read_excel():
    data = pd.read_excel(filepath, sheet_name='建模数据',index_col=[0,1],skiprows=1,usecols=range(0,12))
    return data

data = read_excel()
data.head(9)

fubon_npl  gdp_cn_lg  gdp_cn_yoy  export_gr_yoy  energy_cspt_lg  \
年份     季度                                                                     
2013.0 4.0   0.008206  -1.182989   -0.278078       0.129075       -1.264861   
2014.0 1.0   0.011958  -1.949924   -0.301597      -0.683967       -1.409404   
       2.0   0.014252  -1.502567   -0.289838      -0.374579       -1.255767   
       3.0   0.009055  -1.317760   -0.338184      -0.072387       -1.103768   
       4.0   0.008564  -0.875369   -0.327730      -0.000436       -0.953374   
2015.0 1.0   0.011251  -1.650291   -0.352557      -0.101167       -1.102861   
       2.0   0.009885  -1.203301   -0.353863      -0.374579       -1.026569   
       3.0   0.010819  -1.023952   -0.372156      -0.576041       -0.950682   
       4.0   0.009951  -0.583818   -0.378690      -0.647991       -0.875198   

              oil_rt  coal_cspt_lg  gas_cspt_lg  coal_cspt_yoy  oil_cspt_yoy  
年份     季度                                                                     
2013.0 4.0 -2.163568      0.663034    -1.428546       0.820719      0.080719  
2014.0 1.0 -2.084413     -0.149182    -1.412332       0.541832     -0.007009  
       2.0 -2.005258      0.092671    -1.310147       0.186885     -0.182463  
       3.0 -1.926103      0.326901    -1.209225      -0.161724     -0.357918  
       4.0 -1.846948      0.553609    -1.109534      -0.510332     -0.517423  
2015.0 1.0 -1.411596     -0.360665    -1.116172      -0.637099      0.519356  
       2.0 -0.976244     -0.478400    -1.057427      -1.087121      1.117498  
       3.0 -0.540892     -0.600579    -0.999119      -1.524467      1.715640  
       4.0 -0.105540     -0.727232    -0.941242      -1.961812      2.297831

In [119]:
# 修改data的index为时间序列
start_q = datetime.datetime.strptime(str(data.index[0][0])+'-03-31','%Y.0-%m-%d').date()
data.index = pd.date_range(start_q,periods=len(data),freq='Q-DEC')

In [120]:
data.head(9)

,fubon_npl,gdp_cn_lg,gdp_cn_yoy,export_gr_yoy,energy_cspt_lg,oil_rt,coal_cspt_lg,gas_cspt_lg,coal_cspt_yoy,oil_cspt_yoy
2013-03-31,0.008206,-1.182989,-0.278078,0.129075,-1.264861,-2.163568,0.663034,-1.428546,0.820719,0.080719
2013-06-30,0.011958,-1.949924,-0.301597,-0.683967,-1.409404,-2.084413,-0.149182,-1.412332,0.541832,-0.007009
2013-09-30,0.014252,-1.502567,-0.289838,-0.374579,-1.255767,-2.005258,0.092671,-1.310147,0.186885,-0.182463
2013-12-31,0.009055,-1.317760,-0.338184,-0.072387,-1.103768,-1.926103,0.326901,-1.209225,-0.161724,-0.357918
2014-03-31,0.008564,-0.875369,-0.327730,-0.000436,-0.953374,-1.846948,0.553609,-1.109534,-0.510332,-0.517423
2014-06-30,0.011251,-1.650291,-0.352557,-0.101167,-1.102861,-1.411596,-0.360665,-1.116172,-0.637099,0.519356
2014-09-30,0.009885,-1.203301,-0.353863,-0.374579,-1.026569,-0.976244,-0.478400,-1.057427,-1.087121,1.117498
2014-12-31,0.010819,-1.023952,-0.372156,-0.576041,-0.950682,-0.540892,-0.600579,-0.999119,-1.524467,1.715640
2015-03-31,0.009951,-0.583818,-0.378690,-0.647991,-0.875198,-0.105540,-0.727232,-0.941242,-1.961812,2.297831


In [121]:
# 定义自变量和因变量
X = data.iloc[:,1:data.columns.size]
# Z score standardization
X = (X-X.mean(axis=0))/X.std(axis=0)
Y = np.log(data['fubon_npl']/(1-data['fubon_npl']))

## 2.ADF 检验

### 2.1 自变量ADF检验

In [122]:
result1 = adfuller(data['gdp_cn_lg'].values)
print(result1)

(-1.039333806906561, 0.73864933189139, 8, 25, {'1%': -3.7238633119999998, '5%': -2.98648896, '10%': -2.6328004}, 11.187835183525614)


In [123]:
print('ADF Statistic: %f' % result1[0])
print('p-value: %f' % result1[1])
print('Criticla Values：')
for key, value in result1[4].items():
    print('\t%s: %.3f' % (key,value))
if result1[0] < result1[4]["5%"]:
    print('Fail to reject Null Hypothesis - so the time series is stationary')
else:
    print('Rejected Null Hypothesis - so the time series is non stationary')

ADF Statistic: -1.039334
p-value: 0.738649
Criticla Values：
	1%: -3.724
	5%: -2.986
	10%: -2.633
Rejected Null Hypothesis - so the time series is non stationary


In [124]:
result2 = adfuller(data['gdp_cn_yoy'].values)
print(result2)

(-1.6420231886376953, 0.46120381446905645, 4, 29, {'1%': -3.6790595944893187, '5%': -2.9678817237279103, '10%': -2.6231583472057074}, 61.40486343959045)


In [125]:
result3 = adfuller(data['export_gr_yoy'].values)
print(result3)

(-0.8725996024409416, 0.7968618097266423, 8, 25, {'1%': -3.7238633119999998, '5%': -2.98648896, '10%': -2.6328004}, 53.105966128236936)


In [126]:
result4 = adfuller(data['energy_cspt_lg'].values)
print(result4)

(1.7183907821551814, 0.9981726475288757, 5, 28, {'1%': -3.6889256286443146, '5%': -2.9719894897959187, '10%': -2.6252957653061224}, -44.63579678961426)


In [127]:
result5 = adfuller(data['oil_rt'].values)
print(result5)

(-2.7169434857834696, 0.07115191703073966, 1, 32, {'1%': -3.653519805908203, '5%': -2.9572185644531253, '10%': -2.6175881640625}, -85.47931256954035)


In [128]:
result6 = adfuller(data['coal_cspt_lg'].values)
print(result6)

(1.0303572076532932, 0.994576146409935, 5, 28, {'1%': -3.6889256286443146, '5%': -2.9719894897959187, '10%': -2.6252957653061224}, 10.747413639688439)


In [129]:
result7 = adfuller(data['gas_cspt_lg'].values)
print(result7)

(0.6851722872158181, 0.9895372641734121, 8, 25, {'1%': -3.7238633119999998, '5%': -2.98648896, '10%': -2.6328004}, -79.93748939374026)


In [130]:
result8 = adfuller(data['coal_cspt_yoy'].values)
print(result8)

(-0.030894493844477094, 0.9559013304672457, 1, 32, {'1%': -3.653519805908203, '5%': -2.9572185644531253, '10%': -2.6175881640625}, 18.884347768791145)


In [131]:
result9 = adfuller(data['oil_cspt_yoy'].values)
print(result9)

(-2.719389458610353, 0.07074272294260672, 1, 32, {'1%': -3.653519805908203, '5%': -2.9572185644531253, '10%': -2.6175881640625}, 28.8776020596437)


### 2.2 y值平稳性检验

In [132]:
result10 = adfuller(data['fubon_npl'].values)
print(result10)

(-2.0444691061142386, 0.2674215329996303, 9, 24, {'1%': -3.7377092158564813, '5%': -2.9922162731481485, '10%': -2.635746736111111}, -225.85939450947404)


## 3. 一阶差分

In [133]:
X_diff1 = np.diff(data['gdp_cn_lg'].values)
r_result1 = adfuller (X_diff1)
print(r_result1)

(-1.4345007968426375, 0.5656288575610321, 7, 25, {'1%': -3.7238633119999998, '5%': -2.98648896, '10%': -2.6328004}, 9.92801989287102)


In [134]:
X_diff2 = np.diff(data['gdp_cn_yoy'].values)
r_result2 = adfuller (X_diff2)
print(r_result2)

(-4.740430949534757, 7.060473497306051e-05, 3, 29, {'1%': -3.6790595944893187, '5%': -2.9678817237279103, '10%': -2.6231583472057074}, 61.164126391346635)


In [135]:
X_diff3 = np.diff(data['export_gr_yoy'].values)
r_result3 = adfuller (X_diff3)
print(r_result3)

(-3.09725350268536, 0.026757110368554233, 7, 25, {'1%': -3.7238633119999998, '5%': -2.98648896, '10%': -2.6328004}, 51.74626237767511)


In [136]:
X_diff4 = np.diff(data['energy_cspt_lg'].values)
r_result4 = adfuller (X_diff4)
print(r_result4)

(-0.41823460779578253, 0.9070094445187573, 4, 28, {'1%': -3.6889256286443146, '5%': -2.9719894897959187, '10%': -2.6252957653061224}, -40.76137121057424)


In [137]:
X_diff5 = np.diff(data['oil_rt'].values)
r_result5 = adfuller (X_diff5)
print(r_result5)

(-1.2575861543906481, 0.6483465748214632, 0, 32, {'1%': -3.653519805908203, '5%': -2.9572185644531253, '10%': -2.6175881640625}, -82.18564636173275)


In [138]:
X_diff6 = np.diff(data['coal_cspt_lg'].values)
r_result6 = adfuller (X_diff6)
print(r_result6)

(0.08406513219879291, 0.9649561552904635, 4, 28, {'1%': -3.6889256286443146, '5%': -2.9719894897959187, '10%': -2.6252957653061224}, 13.303455135119215)


In [139]:
X_diff7 = np.diff(data['gas_cspt_lg'].values)
r_result7 = adfuller (X_diff7)
print(r_result7)

(-3.658869191685659, 0.004730333302934265, 7, 25, {'1%': -3.7238633119999998, '5%': -2.98648896, '10%': -2.6328004}, -78.0008255238478)


In [140]:
X_diff8 = np.diff(data['coal_cspt_yoy'].values)
r_result8 = adfuller (X_diff8)
print(r_result8)

(-2.673091044825907, 0.07881155953800628, 0, 32, {'1%': -3.653519805908203, '5%': -2.9572185644531253, '10%': -2.6175881640625}, 17.422146397979958)


In [141]:
X_diff9 = np.diff(data['oil_cspt_yoy'].values)
r_result9 = adfuller (X_diff9)
print(r_result9)

(-3.0672459167813315, 0.02907402275692347, 0, 32, {'1%': -3.653519805908203, '5%': -2.9572185644531253, '10%': -2.6175881640625}, 30.970900594896534)


根据一阶差分后的ADF统计值，发现所有指标的一阶差分至少可在5%的置信水平拒绝单位根假设。

In [142]:
# y值一阶差分平稳
Y_diff = np.diff(data['fubon_npl'].values)
r_result_y = adfuller (Y_diff)
print(r_result_y)

(-1.3501292781737304, 0.6059377685158636, 9, 23, {'1%': -3.7529275211638033, '5%': -2.998499866852963, '10%': -2.6389669754253307}, -221.58437077097403)


In [143]:
X_dif1 = [X_diff1]+ [X_diff2]+ [X_diff3]+ [X_diff4]+ [X_diff5]+ [X_diff6]+ [X_diff7]+ [X_diff8]+ [X_diff9]

In [144]:
X_dif2 = pd.DataFrame(X_dif1)
X_dif2.index = ['gdp_cn_lg', 'gdp_cn_yoy','export_gr_yoy','energy_cspt_lg','oil_rt','coal_cspt_lg','gas_cspt_lg','coal_cspt_yoy','oil_cspt_yoy']
X_dif = X_dif2.T
# 对所有指标进行一阶差分
X_dif

,gdp_cn_lg,gdp_cn_yoy,export_gr_yoy,energy_cspt_lg,oil_rt,coal_cspt_lg,gas_cspt_lg,coal_cspt_yoy,oil_cspt_yoy
0,-0.766935,-0.023520,-0.813042,-0.144543,0.079155,-0.812216,0.016214,-0.278887,-0.087727
1,0.447357,0.011760,0.309388,0.153637,0.079155,0.241853,0.102184,-0.354947,-0.175455
2,0.184807,-0.048346,0.302193,0.151998,0.079155,0.234230,0.100922,-0.348609,-0.175455
3,0.442391,0.010453,0.071951,0.150394,0.079155,0.226708,0.099691,-0.348609,-0.159504
4,-0.774923,-0.024826,-0.100731,-0.149488,0.435352,-0.914274,-0.006638,-0.126767,1.036779
5,0.446990,-0.001307,-0.273412,0.076293,0.435352,-0.117735,0.058745,-0.450022,0.598142
6,0.179349,-0.018293,-0.201462,0.075886,0.435352,-0.122178,0.058308,-0.437345,0.598142
7,0.440134,-0.006533,-0.071951,0.075485,0.435352,-0.126653,0.057877,-0.437345,0.582191
8,-0.211687,1.932528,-0.482069,-0.016809,0.118732,-0.376907,0.048396,0.665526,-0.199381
9,0.464135,0.062719,0.129511,0.095919,0.118732,0.020256,0.080526,0.171135,-0.853349


## 4. 协整关系检验

In [145]:
# 定义协整检验结果输出形式
data_tmp = pd.merge(Y,X,how = 'left',left_index=True,right_index=True)
coint_result = pd.DataFrame(index =[data_tmp.columns],columns=[data_tmp.columns])
# 记录变量间协整检验 p-value
# H0为非协整，p-value越小，拒绝H0证据越充分
for i in range(0,data_tmp.columns.size):
    for j in range(i,data_tmp.columns.size):
        coint_result.iloc[i,j] = coint(data_tmp.iloc[:,i],data_tmp.iloc[:,j],trend='nc',autolag='aic')[1]

print(coint_result)
# 除了dd_rate,其余变量和npl没有两两之间的协整关系

               fubon_npl gdp_cn_lg gdp_cn_yoy export_gr_yoy energy_cspt_lg  \
fubon_npl            0.0  0.900149   0.985828      0.912006       0.979375   
gdp_cn_lg            NaN       0.0   0.979693      0.544569       0.045824   
gdp_cn_yoy           NaN       NaN        0.0      0.311649       0.350053   
export_gr_yoy        NaN       NaN        NaN           0.0       0.001157   
energy_cspt_lg       NaN       NaN        NaN           NaN            0.0   
oil_rt               NaN       NaN        NaN           NaN            NaN   
coal_cspt_lg         NaN       NaN        NaN           NaN            NaN   
gas_cspt_lg          NaN       NaN        NaN           NaN            NaN   
coal_cspt_yoy        NaN       NaN        NaN           NaN            NaN   
oil_cspt_yoy         NaN       NaN        NaN           NaN            NaN   

                  oil_rt coal_cspt_lg gas_cspt_lg coal_cspt_yoy oil_cspt_yoy  
fubon_npl       0.984827     0.839491    0.982014      0.96952

## 5. 建立VAR模型

### 5.1 模型建立

In [146]:
# 使用入模X的一阶差分建立向量自回归模型
# 建立VAR模型
VARmodel = VAR(X_dif)
VARresult = VARmodel.fit(maxlags=2,ic='aic') 
VARresult.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 24, Aug, 2022
Time:                     05:37:13
--------------------------------------------------------------------
No. of Equations:         9.00000    BIC:                   -33.1238
Nobs:                     31.0000    HQIC:                  -38.4553
Log likelihood:           411.140    FPE:                9.24894e-18
AIC:                     -41.0338    Det(Omega_mle):     1.25204e-19
--------------------------------------------------------------------
Results for equation gdp_cn_lg
                       coefficient       std. error           t-stat            prob
------------------------------------------------------------------------------------
const                     1.236596         1.003575            1.232           0.218
L1.gdp_cn_lg             -3.402918         1.392157           -2.444           0.015
L1.gdp_cn_yoy             1.064255      

In [147]:
# 最优滞后阶数
lag_order = VARresult.k_ar
lag_order

2

In [148]:
# 输出残差项
resid = np.array(VARresult.resid)

### 5.2 模型稳定性检验：CUSUM检验

In [149]:
# 模型稳定性检验：CUSUM检验
# 原假设：无漂移（平稳），备择假设：有漂移（不平稳）
result = statsmodels.stats.diagnostic.breaks_cusumolsresid(resid)
print(result)
# (1.351727608700142, 0.05175650227087939, [(1, 1.63), (5, 1.36), (10, 1.22)])
# 可以在5%和1%的置信水平不拒绝原假设，VAR模型平稳

(1.1831793089053038, 0.12161385424909314, [(1, 1.63), (5, 1.36), (10, 1.22)])


In [150]:
#%% MONTE CARLO模拟轻度、中度、重度情景的X：
# 提取残差项的方差协方差矩阵
resid_cov = VARresult.summary().model.resid_corr
# 对残差想的方差协方差矩阵做cholesky decomposition
M = np.linalg.cholesky(resid_cov)
# 验证分解正确 np.mat(M)*np.mat(M.T)-resid_cov

In [151]:
# 入模变量个数
N = X_dif.shape[1]
N

9

In [152]:
# VAR模型的截距项
c = []
for i in range(0,N):
    c.extend([VARresult.summary().model.coefs_exog[i][0]])
c = np.array(c).T

In [153]:
# 构造VAR模型系数估计值矩阵
params = VARresult.summary().model.coefs #(3,10,10)
params_tr_list = []   
for i in range(0,N):
    for j in range(0,lag_order):
        if j == 0:
            tmp = params[j][i] # lag 1 of equation 1
        else:
            tmp = np.hstack([tmp,params[j][i]])
    params_tr_list.extend([tmp])
        
for i in range(0,len(params_tr_list)):
    if i == 0:
        params_tr = params_tr_list[0]
    else:
        params_tr = np.vstack((params_tr,params_tr_list[i]))

In [154]:
# 截取X_dif最近lag_order期的变量
X_dif_lag = X_dif.iloc[-lag_order:,:] #(3,10)
# 准备初始的X_dif
for j in range(0,lag_order):
    if j == 0:
        x_0 = np.array(X_dif_lag.iloc[-1-j,:])
    else:
        x_0 = np.hstack((x_0,np.array(X_dif_lag.iloc[-1-j,:])))
X_dif_lag

,gdp_cn_lg,gdp_cn_yoy,export_gr_yoy,energy_cspt_lg,oil_rt,coal_cspt_lg,gas_cspt_lg,coal_cspt_yoy,oil_cspt_yoy
31,0.417791,-0.041813,-0.223047,0.274068,-0.15831,0.790580,0.138425,0.728909,0.542315
32,-0.765833,-1.469976,-1.014503,0.268897,-0.15831,0.778385,0.133954,0.728909,0.542315


## 7. VECM模型

### 7.1 单变量滞后期与Y的相关性分析

In [155]:
#%% 建立一阶VECM模型，代入模型预测Y
X = data.iloc[:,1:data.columns.size]
Y = np.log(data['fubon_npl']/(1-data['fubon_npl']))
for i in range(1,3):
    X[['L%d_' %(i) + j for j in X_dif.columns]] = np.array(X[X_dif.columns].shift(i))
data_wlag = pd.merge(Y,X,how='left',left_index=True,right_index=True)
corr = data_wlag.corr('spearman')['fubon_npl']
print(corr)
# 当期及滞后期经济变量符号均无问题

fubon_npl            1.000000
gdp_cn_lg           -0.210084
gdp_cn_yoy           0.021850
export_gr_yoy       -0.265128
energy_cspt_lg      -0.258976
oil_rt               0.272359
coal_cspt_lg        -0.772345
gas_cspt_lg         -0.251337
coal_cspt_yoy       -0.301933
oil_cspt_yoy         0.262969
L1_gdp_cn_lg        -0.246324
L1_gdp_cn_yoy        0.074873
L1_export_gr_yoy    -0.198228
L1_energy_cspt_lg   -0.333890
L1_oil_rt            0.076627
L1_coal_cspt_lg     -0.765374
L1_gas_cspt_lg      -0.334225
L1_coal_cspt_yoy    -0.279602
L1_oil_cspt_yoy      0.513412
L2_gdp_cn_lg        -0.262463
L2_gdp_cn_yoy        0.100632
L2_export_gr_yoy    -0.130889
L2_energy_cspt_lg   -0.357038
L2_oil_rt           -0.058131
L2_coal_cspt_lg     -0.745601
L2_gas_cspt_lg      -0.365103
L2_coal_cspt_yoy    -0.365869
L2_oil_cspt_yoy      0.713775
Name: fubon_npl, dtype: float64


### 7.2 遍历所有组合，筛选能通过测试的组合

In [156]:
# 生成各变量经济变量符号
global econ_sign 
econ_sign = pd.DataFrame(data=np.sign(np.array(corr[1:N+1])),index=X_dif.columns)

In [157]:
# 设定各测试阈值；注：此处可修改各测试阈值
lag = 2 ###注：此处可修改滞后期数
paranum = 3 ###注：此处可修改遍历模型入模指标数
#sig_cutoff = 0.5 ###注：此处可修改单变量显著性阈值（各指标和不良率之间相关系数绝对值）
#corr_cutoff = 0.5 ###注：此处可修改相关性筛选阈值（各指标两两智荐相关系数绝对值）
pcutoff = 0.1 ###注：此处可修改长期均衡模型系数p值阈值
R2percentile = 0 ### 注：此处可修改R2阈值，代表分位数（0代表不根据R2筛掉模型）
VIFcutoff = 10 ## 注：此处可修改VIF共线性测试阈值
coint_lag_max = 2 ##注：此处可修改协整测试max lagged-difference in the model
coint_cutoff_map = {0.1:0,0.05:1,0.01:2}
coint_cutoff = 0.1 ##注：此处可修改协整Johansen测试的p-value的阈值 90%，95%，99%，0.1对应90%

In [158]:
# 遍历所有组合
sublst = X.columns.tolist()
sub_comb = []
sub_comb.extend(list(itertools.combinations(sublst,paranum)))
# 从上一步sub_comb中去掉包括指标和其自身滞后期的指标组合
sub_df = pd.DataFrame(index=sub_comb,columns=['comb','no'])
print (len(sub_df))
sub_df['del_lag1'] = sub_df.index
def del_lag(tuples):
    lst = []
    for i in tuples:
        if i[0] == u'L':
            i_new = i[3:]
        else:
            i_new = i
        lst.append(i_new)
        
    tuples_new = tuple(lst)
    set_new = set(tuples_new)
    return set_new
sub_df['del_lag2'] = sub_df[['del_lag1']].applymap(del_lag)
sub_df['del_lag3'] = sub_df[['del_lag2']].applymap(len)
sub_df = sub_df[(sub_df['del_lag3']>=paranum)]
sub_df.drop(['del_lag2','del_lag3'],axis=1,inplace=True)
print (len(sub_df))

2925
2268


### 7.3 回归

In [159]:
# 计算sub_df中模型的回归系数,p统计量，R^2，VIF系数
varlst = []
for i in range(1,paranum+1):
    varlst.append(u'变量' + str(i))
betalst = [u'Intercept']
for i in range(1,paranum+1):
    betalst.append(u'Beta' + str(i))
plst = [u'p0']
for i in range(1,paranum+1):
    plst.append(u'p' + str(i))
regresults_colst = []
regresults_colst.extend(varlst)
regresults_colst.extend(betalst)
regresults_colst.extend(plst)
regresults_colst.append(u'R2')
viflst = []
for i in range(1,paranum+1):
    viflst.append(u'vif_变量' + str(i))
regresults_colst.extend(viflst)

regresults = pd.DataFrame(columns=regresults_colst)

y = Y

In [160]:
# 开始回归
start = time.perf_counter()
for i in sub_df['del_lag1'].tolist():
    print('%d out of %d' %(sub_df['del_lag1'].tolist().index(i)+1,len(sub_df['del_lag1'].tolist())))
    i_lst = list(i)
    x = X[i_lst] 
    #计算vif系数
    x_vif = copy.deepcopy(x)
    x_vif['constant'] = 1
    x_vif = x_vif.dropna()
    vif_values = []
    for j in range(len(i_lst)):
        vif_values.append(variance_inflation_factor(x_vif.loc[x.index[lag]:,i_lst].values,j))
        
    x = sm.add_constant(x)
    model = sm.OLS(y,x,missing='drop')
    results = model.fit()
    if len(i_lst) < paranum:
        gap = copy.deepcopy((paranum-len(i_lst)))
        i_lst.extend([""]*gap)
        reg = pd.DataFrame(np.array(i_lst+results.params.tolist()+[""]*gap+results.pvalues.tolist()+[""]*gap+[results.rsquared,]+vif_values).reshape(1,paranum*4+3),columns=regresults_colst)
    else:
        reg = pd.DataFrame(np.array(i_lst+results.params.tolist()+results.pvalues.tolist()+[results.rsquared,]+vif_values).reshape(1,paranum*4+3),columns=regresults_colst)
    
    regresults = regresults.append(reg,ignore_index=True)
end = time.perf_counter()

1 out of 2268
2 out of 2268
3 out of 2268
4 out of 2268
5 out of 2268
6 out of 2268
7 out of 2268
8 out of 2268
9 out of 2268
10 out of 2268
11 out of 2268
12 out of 2268
13 out of 2268
14 out of 2268
15 out of 2268
16 out of 2268
17 out of 2268
18 out of 2268
19 out of 2268
20 out of 2268
21 out of 2268
22 out of 2268
23 out of 2268
24 out of 2268
25 out of 2268
26 out of 2268
27 out of 2268
28 out of 2268
29 out of 2268
30 out of 2268
31 out of 2268
32 out of 2268
33 out of 2268
34 out of 2268
35 out of 2268
36 out of 2268
37 out of 2268
38 out of 2268
39 out of 2268
40 out of 2268
41 out of 2268
42 out of 2268
43 out of 2268
44 out of 2268
45 out of 2268
46 out of 2268
47 out of 2268
48 out of 2268
49 out of 2268
50 out of 2268
51 out of 2268
52 out of 2268
53 out of 2268
54 out of 2268
55 out of 2268
56 out of 2268
57 out of 2268
58 out of 2268
59 out of 2268
60 out of 2268
61 out of 2268
62 out of 2268
63 out of 2268
64 out of 2268
65 out of 2268
66 out of 2268
67 out of 2268
68 o

In [161]:
print ('回归%d组方程用时%s秒' %(len(sub_df),end-start))

回归2268组方程用时34.112621279999075秒


In [162]:
## 把所有regresults的系数、p值和R^2转化成数值
regresults[betalst+plst+[u'R2']+viflst] = regresults[betalst+plst+[u'R2']+viflst].applymap(lambda x:float(x)) 

In [163]:
# 根据系数经济含义\p统计量\vif小于阈值\以及R^2最大筛选模型 
# 构造regresults新列，去掉L前缀
def delL(string):
    
    if string[0] == u'L':
        string_new = string[3:]
    else:
        string_new = string
   
    return string_new

In [164]:
macro_comb = []    
for i in regresults_colst[0:paranum]:
    i_new = u'还原' + i
    macro_comb.append(i_new)
    regresults[i_new] = regresults[[i]].applymap(delL)
      
def data_pre(row):
    global paranum
    row[u'还原变量组合'] = []
    for i in range(paranum):
        row[u'还原变量组合'].append(row[macro_comb[i]])
    row[u'还原变量组合'] = tuple(row[u'还原变量组合'])
    return row

regresults_dict = regresults.to_dict('records')
for key in regresults_dict:
    data_pre(key)
regresults = pd.DataFrame(regresults_dict)

In [165]:
##导入符合经济含义的系数符号
def signf(x):   
    econ_sign_x = econ_sign.loc[x,0]
    return econ_sign_x

macro_sign = []
for i in macro_comb:
    i_new = i + u'econ_sign'
    macro_sign.append(i_new)
    regresults[i_new] = regresults[[i]].applymap(signf)

In [166]:
# 记录是否有变量系数符号和其经济含义不符，如有标记为0，否则标记为1
regresults['econ_sign_tag'] = 1
for i in regresults.index.tolist():
    cond_lst = []
    for j in range(paranum):
        cond_lst.append(np.sign(float(regresults.loc[i,u'Beta'+str(j+1)])) != regresults.loc[i,macro_sign[j]])
    if 1 in cond_lst:
        regresults.loc[i,'econ_sign_tag'] = 0

In [167]:
# 记录是否有变量显著性测试不能通过，如有标记为0，否则标记为1
regresults['significance_tag'] = 1
for i in regresults.index.tolist():
    cond_lst = []
    for j in range(paranum):
        cond_lst.append(float(regresults.loc[i,u'p'+str(j+1)]) > pcutoff)
    if  1 in cond_lst:
        regresults.loc[i,'significance_tag'] = 0

In [168]:
# 记录是否有变量多重共线性VIF测试不能通过，如有标记为0，否则标记为1
regresults['vif_tag'] = 1
for i in regresults.index.tolist():
    cond_lst = []
    for j in range(paranum):
        cond_lst.append(float(regresults.loc[i,u'vif_变量'+str(j+1)]) > VIFcutoff)
    if 1 in cond_lst:
        regresults.loc[i,'vif_tag'] = 0

In [169]:
print(u'共回归 %d 组模型' %(len(regresults)))

共回归 2268 组模型


In [170]:
## 去掉通不过系数经济含义测试\显著性测试\VIF共线性测试的模型
regresults_sub1 = regresults[((regresults['econ_sign_tag'] == 0) & (regresults['significance_tag'] == 0) & (regresults['vif_tag'] == 0))]
print (u'经济含义\系数显著性\多重共线性测试通过后有 %d 组模型' %(len(regresults_sub1))) #272组

经济含义\系数显著性\多重共线性测试通过后有 180 组模型


### 7.4 长期均衡模型结果输出

In [171]:
## 将模型按照入模宏观指标类型和R^2降序排列
regresults = regresults.sort_values(by=[u'还原变量组合',u'R2'],ascending=False)
regresults_sub1 = regresults_sub1.sort_values(by=[u'还原变量组合',u'R2'],ascending=False)

In [172]:
## 考察R^2的分位数
regresults[[u'R2']].describe(include='all',percentiles=[0,0.25,0.5,0.75,1])

,R2
count,2268.000000
mean,0.438232
std,0.152519
min,0.082872
0%,0.082872
25%,0.304486
50%,0.460056
75%,0.556763
100%,0.798644
max,0.798644


In [173]:
## 可以选择100%分位数
print (np.percentile(regresults[u'R2'],R2percentile))
regresults_sub2 = regresults_sub1[regresults_sub1[u'R2'] > np.percentile(regresults[u'R2'],R2percentile)]
print (u'保留%d%%R2后有%d组模型' %(R2percentile,len(regresults_sub2)))

0.08287248547417458
保留0%R2后有180组模型


In [174]:
regresults_sub2.to_excel(r'output_long_term_model_8.24_滞后期数2.xlsx')

### 7.5 协整测试结果输出

In [175]:
# 对保留的模型做协整关系测试(Johansen测试)，保留通过协整测试的模型
# 这一步之前首先人工筛选，缩小入选模型的范围
model_nobs_lst = regresults_sub2.index.tolist() ## 注：此处可修改入选模型编号 [1002,997]
model_nobs_lst_keep = [] #此处保存通过协整测试的模型编号
model_nobs_lst_keep = copy.deepcopy(model_nobs_lst)
regresults_sub3 = regresults.loc[model_nobs_lst]

In [176]:
regresults_sub3['coint_tag'] = 0
regresults_sub3['coint_rank'] = 0

In [177]:
model_col_lst = []
for i in range(1,paranum+1):
    model_col_lst.append(u'变量'+str(i))

In [178]:
for i in model_nobs_lst:
    model_varlst = regresults_sub3.loc[i,model_col_lst].tolist()
    x = pd.merge(Y,X[model_varlst],how='left',left_index=True,right_index=True).dropna()
    coint_results = coint_johansen(x, 1, coint_lag_max)
    # print (coint_results.cvt)   # 跟踪统计的临界值
    # print (coint_results.lr1)  # 跟踪统计
    coint_results_cvt = []
    for j in range(paranum+1):
        coint_results_cvt.append(coint_results.cvt[j][coint_cutoff_map[coint_cutoff]])
    # print (coint_results_cvt)
    for j in range(paranum+1):
        if j == 0 and coint_results.lr1[j] <= coint_results_cvt[j]:
            regresults_sub3.loc[i,'coint_rank'] = 0
            break
        else:
            if coint_results.lr1[j] > coint_results_cvt[j]:
                regresults_sub3.loc[i,'coint_rank'] += 1
                continue
            else:
                break
    if regresults_sub3.loc[i,'coint_rank'] >= 3:
        regresults_sub3.loc[i,'coint_tag'] = 1
    if regresults_sub3.loc[i,'coint_rank'] == 4:
        regresults_sub3.loc[i,'coint_rank'] = '> 3'
    if regresults_sub3.loc[i,'coint_tag'] == 1:
        model_nobs_lst_keep.append(i)

In [179]:
## 协整测试结果输出
regresults_sub3 = regresults_sub3[regresults_sub3['coint_tag']==1]
regresults_sub3.to_excel(r'output_cointegration_test_8.24_滞后期数2.xlsx')

## 8. 对最终选定的模型，使用VECM模型，估计短期修正模型

In [180]:
#从上一步保留的模型池中选定最终使用的模型
model_no = 727 #注：此处修改最终选定的模型编号
regresults_sub4 = pd.DataFrame(regresults_sub3.loc[model_no,:]).T
#构造各变量的一阶差分
model_varlst = regresults_sub4[model_col_lst].values[0].tolist() + [u'fubon_npl']
d_model_varlst = [u'D_' + i for i in model_varlst] + [u'fubon_npl_p',u'ecm']
d_model_varlst_sub = [u'D_' + i for i in regresults_sub4[model_col_lst].values[0].tolist()] + [u'ecm']
d_value = pd.DataFrame(columns=d_model_varlst)
data_tmp = pd.merge(Y,X,how='left',left_index=True,right_index=True)
for i in range(paranum+1):
    d_value[d_model_varlst[i]] = data_tmp[model_varlst[i]].diff(1)
d_value[u'fubon_npl_p'] = regresults_sub4.loc[model_no,u'Intercept']
for i in range(paranum):
    d_value[u'fubon_npl_p'] += regresults_sub4.loc[model_no,u'Beta'+str(i+1)]*data_tmp[model_varlst[i]]
tmp = data_tmp[u'fubon_npl'] - d_value[u'fubon_npl_p']
d_value[u'ecm'] = (data_tmp[u'fubon_npl'] - d_value[u'fubon_npl_p']).shift(1)

KeyError: ignored

In [ ]:
#回归短期误差修正模型
x = d_value[d_model_varlst_sub]
x = sm.add_constant(x)
model = sm.OLS(d_value['D_fubon_npl'],x.astype(float),missing='drop')
results_tmp = model.fit()

d_col_lst = [u'D_Intercept']
for i in d_model_varlst_sub:
    d_col_lst.append(u'Beta_'+i)
for i in d_col_lst:
    regresults_sub4.loc[model_no,i] = 0

regresults_sub4[d_col_lst] = results_tmp.params.tolist()

In [ ]:
##短期修正模型结果输出
regresults_sub4.to_excel('output_ecm_model.xlsx')